In [1]:
pip show mlflow


Name: mlflow
Version: 3.8.1
Summary: MLflow is an open source platform for the complete machine learning lifecycle
Home-page: 
Author: 
Author-email: 
License: Copyright 2018 Databricks, Inc.  All rights reserved.

                                Apache License
                           Version 2.0, January 2004
                        http://www.apache.org/licenses/

   TERMS AND CONDITIONS FOR USE, REPRODUCTION, AND DISTRIBUTION

   1. Definitions.

      "License" shall mean the terms and conditions for use, reproduction,
      and distribution as defined by Sections 1 through 9 of this document.

      "Licensor" shall mean the copyright owner or entity authorized by
      the copyright owner that is granting the License.

      "Legal Entity" shall mean the union of the acting entity and all
      other entities that control, are controlled by, or are under common
      control with that entity. For the purposes of this definition,
      "control" means (i) the power, direct or i

In [2]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [3]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
742,"Never heard of this movie,saw it on DVD.Great ...",positive
539,I really wanted to like The Pillow Book. Intri...,negative
854,"I normally am a glutton for poor movies, but t...",negative
686,"Firstly,I must admit that it isn't a good movi...",negative
740,So I'm looking to rent a DVD and I come across...,negative


In [4]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [5]:
df = normalize_text(df)
df.head()

,review,sentiment
742,never heard movie saw dvd great movie perfect ...,positive
539,really wanted like pillow book intriguing stor...,negative
854,normally glutton poor movie thing stunk br br ...,negative
686,firstly i must admit good movie and i would ne...,negative
740,looking rent dvd come across movie called end ...,negative


In [6]:
df['sentiment'].value_counts()

sentiment
positive    251
negative    249
Name: count, dtype: int64

In [7]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [8]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
742,never heard movie saw dvd great movie perfect ...,1
539,really wanted like pillow book intriguing stor...,0
854,normally glutton poor movie thing stunk br br ...,0
686,firstly i must admit good movie and i would ne...,0
740,looking rent dvd come across movie called end ...,0


In [9]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [17]:
vectorizer = CountVectorizer(max_features=50)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [19]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/shekhus/capastan_sentiment_analysis.mlflow')
dagshub.init(repo_owner='shekhus', repo_name='capastan_sentiment_analysis', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


2026-01-11 15:29:46,232 - INFO - HTTP Request: GET https://dagshub.com/api/v1/repos/shekhus/capastan_sentiment_analysis "HTTP/1.1 200 OK"


Initialized MLflow to track repo "shekhus/capastan_sentiment_analysis"

2026-01-11 15:29:46,247 - INFO - Initialized MLflow to track repo "shekhus/capastan_sentiment_analysis"


Repository shekhus/capastan_sentiment_analysis initialized!

2026-01-11 15:29:46,252 - INFO - Repository shekhus/capastan_sentiment_analysis initialized!


<Experiment: artifact_location='mlflow-artifacts:/b4db7a95e18946cba78671b3ac093502', creation_time=1768124267916, experiment_id='0', last_update_time=1768124267916, lifecycle_stage='active', name='Logistic Regression Baseline', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [20]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 50)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2026-01-11 15:29:50,874 - INFO - Starting MLflow run...


2026-01-11 15:29:51,293 - INFO - Logging preprocessing parameters...
2026-01-11 15:29:52,350 - INFO - Initializing Logistic Regression model...
2026-01-11 15:29:52,353 - INFO - Fitting the model...
2026-01-11 15:29:52,427 - INFO - Model training complete.
2026-01-11 15:29:52,431 - INFO - Logging model parameters...
2026-01-11 15:29:52,801 - INFO - Making predictions...
2026-01-11 15:29:52,806 - INFO - Calculating evaluation metrics...
2026-01-11 15:29:52,831 - INFO - Logging evaluation metrics...
2026-01-11 15:29:54,157 - INFO - Saving and logging the model...
2026/01/11 15:29:54 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026-01-11 15:30:18,150 - INFO - Model training and logging completed in 26.86 seconds.
2026-01-11 15:30:18,152 - INFO - Accuracy: 0.656
2026-01-11 15:30:18,154 - INFO - Precision: 0.78125
2026-01-11 15:30:18,155 - INFO - Recall: 0.6329113924050633
2026-01-11 15:30:18,158 - INFO - F1 Score: 0.6993006993006993


🏃 View run lyrical-snake-792 at: https://dagshub.com/shekhus/capastan_sentiment_analysis.mlflow/#/experiments/0/runs/544ebade64d94ccba28d3ba00e6025fe
🧪 View experiment at: https://dagshub.com/shekhus/capastan_sentiment_analysis.mlflow/#/experiments/0
